In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
import scipy as sp
import os
from scipy.linalg import fractional_matrix_power

from jax import vmap, jit, numpy as jnp, random, lax, jvp, scipy as jsp
import matplotlib.pyplot as plt
import pickle
from functools import reduce

import matplotlib.animation as animation
from IPython.display import HTML

module_path = os.path.abspath(os.path.join("/Users/shufay/Documents/ad_afqmc"))
if module_path not in sys.path:
    sys.path.append(module_path)

from ad_afqmc import (
    driver,
    pyscf_interface,
    mpi_jax,
    linalg_utils,
    spin_utils,
    lattices,
    propagation,
    wavefunctions,
    hamiltonian,
    hf_guess
)

from pyscf import fci, gto, scf, mp, ao2mo

import itertools
from functools import partial

np.set_printoptions(precision=5, suppress=True)

# Hostname: g152
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64


In [70]:
lattice = lattices.triangular_grid(6, 6, open_x=True)
n_sites = lattice.n_sites
u = 12.0
n_elec = (18, 18)
nocc = sum(n_elec)

integrals = {}
integrals["h0"] = 0.0

h1 = -1.0 * lattice.create_adjacency_matrix()
integrals["h1"] = h1

h2 = np.zeros((n_sites, n_sites, n_sites, n_sites))
for i in range(n_sites):
    h2[i, i, i, i] = u
integrals["h2"] = ao2mo.restore(8, h2, n_sites)

# make dummy molecule
mol = gto.Mole()
mol.nelectron = nocc
mol.incore_anyway = True
mol.spin = abs(n_elec[0] - n_elec[1])
mol.build()

# ghf
gmf = scf.GHF(mol)
gmf.get_hcore = lambda *args: sp.linalg.block_diag(integrals["h1"], integrals["h1"])
gmf.get_ovlp = lambda *args: np.eye(2 * n_sites)
gmf._eri = ao2mo.restore(8, integrals["h2"], n_sites)

# construct Neel guess.
psi0 = hf_guess.get_ghf_neel_guess(lattice)
dm_init = psi0[:, :nocc] @ psi0[:, :nocc].T.conj()
# dm_init = np.random.random((2*n_sites, 2*n_sites))
# dm_init += dm_init.T.conj()

gmf.kernel(dm_init)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)

gmf_coeff = gmf.mo_coeff
gmf_rdm1 = gmf.make_rdm1()
 
epsilon0, mu, spin_axis = spin_utils.spin_collinearity_test(gmf_coeff[:, :nocc], ao_ovlp, verbose=1)

converged SCF energy = -11.6658907088659  <S^2> = 17.555108  2S+1 = 8.4392199
<class 'pyscf.scf.ghf.GHF'> wavefunction is stable in the internal stability analysis
converged SCF energy = -11.6658907104589  <S^2> = 17.555203  2S+1 = 8.4392423
<class 'pyscf.soscf.newton_ah.SecondOrderGHF'> wavefunction is stable in the internal stability analysis
converged SCF energy = -11.6658907107999  <S^2> = 17.555283  2S+1 = 8.4392615
<class 'pyscf.soscf.newton_ah.SecondOrderGHF'> wavefunction is stable in the internal stability analysis
converged SCF energy = -11.6658907108129  <S^2> = 17.555284  2S+1 = 8.4392615
<class 'pyscf.soscf.newton_ah.SecondOrderGHF'> wavefunction is stable in the internal stability analysis

# ----------------------
# Spin collinearity test
# ----------------------
# epsilon0 = 0.9483331390700681
# non integer value indicates non-collinearity

# minimum mu = 3.70343009422942
# Value is 0 iff wavefunction is collinear

# If mu = 0, the collinear spin axis is: 
[-0.  0.  1.]

/burg/ccce/users/su2254/ad_afqmc/ad_afqmc/spin_utils.py:184: ComplexWarning: Casting complex values to real discards the imaginary part
  tril[i, j] = -np.trace(O_mats[i] @ O_mats[j])


In [69]:
# fci
if sum(n_elec) < 8:
    ci = fci.FCI(mol)
    e, ci_coeffs = ci.kernel(
        h1e=integrals["h1"], eri=integrals["h2"], norb=n_sites, nelec=n_elec
    )
    print(f"fci energy: {e}")

pyscf_interface.prep_afqmc(
        gmf, basis_coeff=np.eye(n_sites), integrals=integrals)

options = {
    "dt": 0.005,
    "n_eql": 10,
    "n_ene_blocks": 1,
    "n_sr_blocks": 5,
    "n_blocks": 100,
    "n_prop_steps": 50,
    "n_walkers": 40,
    "seed": 98,
    "walker_type": "ghf",
    "save_walkers": False,
}

run_cpmc = True
verbose = 1

ham_data, ham, prop, trial, wave_data, sampler, observable, options, MPI = (
    mpi_jax._prep_afqmc(options)
)

if run_cpmc:
    if verbose: print(f'\n# Using CPMC propagator with GHF walkers...')
    prop = propagation.propagator_cpmc_general(
        dt=options["dt"],
        n_walkers=options["n_walkers"],
    )

trial = wavefunctions.ghf_cpmc(n_sites, n_elec)

# Overwrite wave_data fields.
wave_data["mo_coeff"] = gmf_coeff[:, :nocc]
wave_data["rdm1"] = gmf_rdm1

# Overwrite ham_data.
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, prop, trial, wave_data)
ham_data["u"] = u
init_walkers = jnp.array([wave_data["mo_coeff"]] * options["n_walkers"])

e_afqmc, err_afqmc = driver.afqmc(
    ham_data, ham, prop, trial, wave_data, sampler, observable, options, MPI, init_walkers=init_walkers
)

#
# Preparing AFQMC calculation
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (18, 18)
# Number of basis functions: 36
# Number of Cholesky vectors: 36
#
# Number of MPI ranks: 1
#
# No trial specified in options.
# trial.pkl not found, make sure to construct the trial separately.
# Propagator not constructed. Make sure to construct it separately.
# norb: 36
# nelec: (18, 18)
#
# dt: 0.005
# n_eql: 10
# n_ene_blocks: 1
# n_sr_blocks: 5
# n_blocks: 100
# n_prop_steps: 50
# n_walkers: 40
# seed: 98
# walker_type: ghf
# save_walkers: False
# n_ene_blocks_eql: 5
# n_sr_blocks_eql: 10
# orbital_rotation: True
# do_sr: True
# symmetry: False
# ene0: 0.0
# free_projection: False
# n_batch: 1
#

# Using CPMC propagator with GHF walkers...
# Equilibration sweeps:
#   Iter        Block energy      Walltime
#     0      -1.166589071e+01     6.65e-01 
#     1      -1.252382755e+01     6.52e+01 
#     2      -1.2

In [ ]:
# rotate ghf coefficients
gmf_coeff = np.zeros((2 * n_sites, 2 * n_sites))
args = sys.argv
theta = float(args[1])
gmf_coeff[:n_sites, :] = (
 np.cos(theta / 2) * gmf.mo_coeff[:n_sites, :]
 - np.sin(theta / 2) * gmf.mo_coeff[n_sites:, :]
)
gmf_coeff[n_sites:, :] = (
 np.sin(theta / 2) * gmf.mo_coeff[:n_sites, :]
 + np.cos(theta / 2) * gmf.mo_coeff[n_sites:, :]
)
gmf.mo_coeff = gmf_coeff